# Import Dependencies..

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# import tensorflow as tf..
from tensorflow import keras

In [4]:
from keras.layers import Input, Lambda, Dense, Flatten

# Using TensorFlow Backend...

In [5]:
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input    #Visual Geometry Group, with 16 Convolutional Layers...
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np

# In Python, the 'glob' module is used similarly to find, locate, and search, for all of the files that are present in a system..
from glob import glob

import matplotlib.pyplot as plt

# Define the paths for Training and Test datasets...

In [6]:
IMAGE_SIZE = [224, 224]

train_path = "E:\Pneumonia Detection using X- Ray Images\chest x-ray images\train"
test_path = "E:\Pneumonia Detection using X- Ray Images\chest x-ray images\test"

In [8]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights="imagenet", include_top=False)

In [9]:
for layer in vgg.layers:
    layer.trainable = False

# In TensorFlow, setting "layer.trainable = False" freezes a layer by preventing its weights from being updated during training. This is also known as "freezing" the layer...

In [10]:
folders = glob("chest x-ray images/train/*")
x = Flatten()(vgg.output)

In [11]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object...
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model...
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

# Importing ImageDataGenerator

In [14]:
from keras.preprocessing.image import ImageDataGenerator

In [15]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

# Make sure you provide the same target size as initialized for the Image size...

In [16]:
training_set = train_datagen.flow_from_directory("chest x-ray images/train",
                                                target_size = (224, 224),
                                                batch_size = 10,
                                                class_mode = 'categorical')

test_set = test_datagen.flow_from_directory("chest x-ray images/test",
                                           target_size = (224, 224),
                                           batch_size = 10,
                                           class_mode = 'categorical')


print(len(training_set), len(test_set))

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
522 63


In [17]:
r = model.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 1,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)



522/522 [==============================] - 702s 1s/step - loss: 0.2030 - accuracy: 0.9312 - val_loss: 0.6987 - val_accuracy: 0.8093


In [18]:
import tensorflow as tf
from keras.models import load_model

model.save('chest_xray.h5')

In [19]:
from keras.models import load_model

In [20]:
from keras.preprocessing import image

In [21]:
from keras.applications.vgg16 import preprocess_input

In [23]:
import numpy as np

In [23]:
model = load_model('chest_xray.h5')

In [24]:
img = image.load_img("E:\\Pneumonia Detection using X- Ray Images\\chest x-ray images\\val\\PNEUMONIA\\person1947_bacteria_4876.jpeg", target_size = (224,224))

In [25]:
x = image.img_to_array(img)

In [26]:
x = np.expand_dims(x, axis=0)

In [27]:
img_data = preprocess_input(x)

In [28]:
classes = model.predict(img_data)

1/1 [==============================] - 1s 593ms/step


In [29]:
classes

# for normal folder, index will be 1..
# for pneumonia folder, index will be 0..

array([[0., 1.]], dtype=float32)

In [30]:
result = classes[0][0]
result

0.0

In [31]:
if(result > 0.5):
    print("Result is Normal")
else:
    print("Person is affected by pneumonia")

Person is affected by pneumonia
